Guidelines:

•	introduce the context, what are the exonerations documents, why do we want to index officers
•	very briefly explain where we are in the process (e.g. we’ve done page classification, we have these transcripts, etc)

•	introduce the extraction method – do we want to address alternative approaches? 
    Maybe we discuss a regex-based solution just as a way of introducing our method and the strengths/weaknesses
•	describe the main problem: responses to multiple prompts, no automatic way to choose the best response
•	describe our solution with the summarizer

Exoneration documents—records that formally vindicate individuals erroneously convicted of crimes—serve as rich, informative resources in the field of wrongful conviction research. These documents are particularly revealing about the law enforcement personnel involved in such cases. However, deciphering these documents and organizing the officers' roles is a challenging task, given their unstructured and extensive nature.

In our effort to unravel these complexities and bring transparency to the criminal justice system, we've developed a multi-stage process:

1 - Metadata Compilation: We started by compiling a comprehensive CSV index that includes metadata such as file paths, names, hash values, sizes, unique identifiers, file types, and case IDs. This structured approach forms the foundation of our file management system, enabling efficient file retrieval.

2 - File Identification: Next, we identified key case files by probing filenames for specific search terms. This procedure allowed us to zero in on essential documents like police reports, testimonies, and wrongful conviction transcripts.

3 - Image Classification: To process these documents, we turned to FastAI—a robust deep learning library. We fine-tuned a pre-trained Convolutional Neural Network (CNN) to identify reports, transcripts, and testimonies based on visual cues. This streamlined approach considerably enhances our system's efficiency and effectiveness.

4 - Information Extraction: Currently, we're engaged in extracting structured information from the documents we've identified. We use a combination of natural language processing libraries—including Chat-GPT, Langchain, regular expressions, and Microsoft Form Recognizer—to gather critical data about police officers and prosecutors, such as their names, ranks, and roles.

5 - Deduplication: After information extraction, we refine our index to ensure that each officer corresponds to a single row of data. This cleanup stage is essential to maintain the accuracy and comprehensiveness of our index.

6 - Cross-referencing: In the final stage, we'll cross-reference the officer names and roles we've extracted with the Louisiana Law Enforcement Accountability Database (LLEAD). This critical step will help us identify additional individuals associated with implicated officers, and allow us to request public records, thus uncovering the identities of all individuals arrested by these officers.

A primary task in our process is extracting officer information from documents. We initially considered a regex-based solution but soon realized that the complexity and variability of the data rendered regex less than ideal. While regex is efficient for pattern matching, it struggles with language variations and nuances.

For example, let's look at the task of cleaning officer names. Here's an illustration of how we employ regex in our script:

In [ ]:
def clean_name(officer_name):
    return re.sub(
        r"(Detective|Officer|Deputy|Captain|[CcPpLl]|Sergeant|Lieutenant|Techn?i?c?i?a?n?)\\.?\\s+",
        "",
        officer_name,
    )

This function aims to remove titles from officer names. It uses a regular expression to match common titles (like "Detective", "Officer", "Deputy", etc.) and remove them. However, this approach can falter if the titles do not exactly match the regex pattern.

To resolve this issue, we turned to AI—specifically Langchain, a proprietary natural language processing library, and OpenAI's language model, GPT-3. Using GPT-3 allows us to handle a wider variety of data and extract more nuanced information.

For example, consider the method we use to generate hypothetical document embeddings. These embeddings encapsulate the contextual information in our documents.

In [ ]:
def generate_hyde():
    llm = OpenAI()
    prompt_template = """\
    You're an AI assistant specializing in criminal justice research. 
    Your main focus is on identifying the names and providing detailed context of mention for each law enforcement personnel.
    ...
    """
    prompt = PromptTemplate(input_variables=["question"], template=prompt_template)
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    base_embeddings = OpenAIEmbeddings()
    embeddings = HypotheticalDocumentEmbedder(llm_chain=llm_chain, base_embeddings=base_embeddings)
    return embeddings

This function initializes an instance of the OpenAI language model, defines a prompt template for the language model, and creates a hypothetical document embedder instance with the language model and the prompt. The hypothetical document embedder is used to generate embeddings for the documents we process.

Our analysis of wrongful conviction cases employs a suite of computational tools: the RecursiveCharacterTextSplitter, FAISS (Facebook AI Similarity Search), a structured PromptTemplate, and the gpt-3.5-turbo-0613 model.

The RecursiveCharacterTextSplitter initiates the text processing stage. It segments extensive textual data from case files into manageable chunks. The configured chunk size of 3000 characters and overlap of 1500 characters ensure preservation of contextual information across divisions.

Following the text segmentation, we utilize FAISS for document similarity search. This library, developed by Meta's AI Research group, interprets each chunk as a vector, enabling efficient identification of chunks most similar to a given query, thus facilitating efficient analysis of large corpus.

Subsequent to the document similarity search, our structured PromptTemplate comes into play, formulating precise and consistent queries. These queries are processed by the gpt-3.5-turbo-0613 model This model generates responses by extracting pertinent information from our document chunks.

Each tool serves a distinct yet interconnected role in the information extraction process. The RecursiveCharacterTextSplitter prepares our data, FAISS optimizes the search within this data, and the integration of our PromptTemplate with gpt-3.5-turbo-0613 enables effective data interpretation. Collectively, they form a comprehensive pipeline that addresses the challenges associated with large-scale data analysis in wrongful conviction cases.

In [ ]:
def process_single_document(file_path, embeddings):
    logger.info(f"Processing document: {file_path}")

    loader = Docx2txtLoader(file_path)
    text = loader.load()
    logger.info(f"Text loaded from document: {file_path}")

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=1500)
    docs = text_splitter.split_documents(text)

    db = FAISS.from_documents(docs, embeddings)

    return db

def get_response_from_query(db, query, k=3):
    logger.info("Performing query...")
    docs = db.similarity_search(query, k=k)
    docs_page_content = " ".join([d.page_content for d in docs])

    llm = ChatOpenAI(model_name="gpt-3.5-turbo-0613")

    prompt = PromptTemplate(
        input_variables=["question", "docs"],
        template="""
        As an AI assistant, my role is to meticulously analyze court transcripts and extract information about law enforcement personnel.
        Note: titles "Technician" and "Tech" might be used interchangeably.
        Note: titles may be abbreviated to the following Sgt., Cpl, Cpt, Det., Ofc., Lt., P.O. and P/O

        Query: {question}

        Transcripts: {docs}

        The response will contain:

        1) The name of a police officer, detective, deputy, lieutenant, 
           sergeant, captain, officer, coroner, investigator, criminalist, or technician.
           If the name does not have one of these titles as a prefix, the person does not work in law enforcement.  
           If you are not at least fifty percent sure that the identified individual works in law enforcement, 
           add "Based on the name, I'm not sure that this individual works in law enforcement".
           Please prefix the name with "Officer Name: ". 
           For example, "Officer Name: John Smith".

        2) If available, provide an in-depth description of the context of their mention. 
           If the context induces ambiguity regarding the individual's employment in law enforcement, 
           add "Based on the context, I'm not sure that this individual works in law enforcement".
           Please prefix this information with "Officer Context: ". 

        Continue this pattern for each officer and each context, until all law enforcement personnel are identified.  

        Guidelines for the AI assistant:
        
        - Derive responses from factual information found within the police reports.
        - If the context of an identified person's mention is not clear in the report, provide their name and note that the context is not specified.
        - If there is insufficient information to answer the query, simply respond with "Insufficient information to answer the query".
    """,
    )

    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(question=query, docs=docs_page_content, temperature=0)

    formatted_response = ""
    officers = response.split("Officer Name:")
    for i, officer in enumerate(officers):
        if officer.strip() != "":
            formatted_response += f"Officer Name {i}:{officer.replace('Officer Context:', 'Officer Context ' + str(i) + ':')}\n\n"

    officer_data = extract_officer_data(formatted_response)
    return officer_data, docs


queries = [
    "Identify individuals with the specific titles of police officers, sergeants, lieutenants, captains, detectives, homicide officers, and crime lab personnel in the transcript. Provide the context of their mention, if available.",
    "List individuals specifically titled as police officers, sergeants, lieutenants, captains, detectives, homicide units, and crime lab personnel mentioned in the transcript and provide the context of their mention.",
    "Locate individuals specifically referred to as police officers, sergeants, lieutenants, captains, detectives, homicide units, and crime lab personnel in the transcript and explain their context of mention.",
    "Highlight individuals explicitly titled as police officers, sergeants, lieutenants, captains, detectives, homicide units, and crime lab personnel in the transcript and describe their context of mention.",
    "Outline individuals directly identified as police officers, sergeants, lieutenants, captains, detectives, homicide units, and crime lab personnel in the transcript and specify their context of mention.",
    "Pinpoint individuals directly labeled as police officers, sergeants, lieutenants, captains, detectives, homicide units, and crime lab personnel in the transcript and provide their context of mention.",
]

Despite the strengths of AI, a major challenge remains: determining the best response from the AI model. Given a prompt, the model can yield multiple responses, and figuring out which response is the most accurate or relevant is not straightforward.

Let's consider a situation where we have multiple queries, and for each query, an officer may be identified more than once. This repetition is not a limitation but an inherent characteristic of our approach because it allows us to capture every possible mention of an officer. Hence, we end up with a rich, albeit redundant, dataset, where the same officer could be mentioned multiple times across different queries.

To handle this redundancy and to extract the most valuable information, we use a summarizer model. The summarizer model takes as input the multiple responses from the AI model. It then condenses them and extracts the most crucial information, providing a summary that amalgamates the information from all responses.

This summary doesn't just reduce the length of the text; it also synthesizes the multiple mentions of each officer, helping us understand the different contexts in which they were mentioned. As a result, we get a consolidated, comprehensive view of each officer's involvement in the cases we're examining.

In [ ]:
def summarize_context(context):
    model = Summarizer()
    result = model(context, min_length=60)
    summary = "".join(result)
    return summary

Through this method, we can handle the multi-dimensional nature of our data—multiple queries, multiple documents, and multiple mentions of each officer—and distill it into a coherent and concise summary. This summary forms the basis of our index, ensuring that we capture a comprehensive picture of each officer's role in the exoneration cases.